In [ ]:
!wget "https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2024/mono/en.txt.gz"
!gunzip en.txt.gz

--2025-04-30 07:22:09--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2024/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9508584209 (8.9G) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>]   8.86G  29.4MB/s    in 5m 17s  

2025-04-30 07:27:28 (28.6 MB/s) - ‘en.txt.gz’ saved [9508584209/9508584209]



In [ ]:

import pandas as pd
import torch
from transformers import pipeline
import os
import glob
import shutil


os.makedirs('/content/split_files', exist_ok=True)
os.makedirs('/content/translated', exist_ok=True)
# Drive folder for checkpoints
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/translation_checkpoints', exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

def add_translated_splits_to_log(translation_dir='/content/translated'):
    # Log file where completed splits are stored
    completed_log = '/content/translated/log_completed.txt'

    # Get the list of already translated split files in the translation directory
    translated_files = [f for f in os.listdir(translation_dir) if f.endswith('_translated.csv')]

    # Extract base names from the translated files (remove the '_translated.csv' part)
    translated_splits = {os.path.splitext(f)[0].replace('DS4', '') for f in translated_files}

    # Read the existing log file to get already recorded completed splits
    if os.path.exists(completed_log):
        with open(completed_log, 'r') as log_file:
            already_logged_splits = {line.strip() for line in log_file.readlines()}
    else:
        already_logged_splits = set()

    # Combine the already logged splits with the new translated ones
    new_splits_to_add = translated_splits - already_logged_splits

    # If there are any new splits to add, append them to the log file
    if new_splits_to_add:
        with open(completed_log, 'a') as log_file:
            for split in new_splits_to_add:
                log_file.write(f"{split}\n")
        print(f"✅ Added {len(new_splits_to_add)} new splits to the log.")
    else:
        print("No new splits to add to the log.")

# Run this function to add the already translated splits to the log file
add_translated_splits_to_log()


✅ Added 1 new splits to the log.


In [ ]:

language_models = {
    'Chinese': 'Helsinki-NLP/opus-mt-en-zh',
    'Spanish': 'Helsinki-NLP/opus-mt-en-es',
    'French': 'Helsinki-NLP/opus-mt-en-fr',
    'German': 'Helsinki-NLP/opus-mt-en-de',
    'Italian': 'Helsinki-NLP/opus-mt-en-it',
    'Hindi': 'Helsinki-NLP/opus-mt-en-hi',
    'Urdu': 'Helsinki-NLP/opus-mt-en-ur',
    'Arabic': 'Helsinki-NLP/opus-mt-en-ar',
}

# Determine if GPU is available for faster processing
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
translation_pipelines = {}

print("🔵 Loading translation models...")
for lang, model_name in language_models.items():
    translation_pipelines[lang] = pipeline('translation', model=model_name, tokenizer=model_name, device=device)
print("✅ Models loaded successfully!")

def translate_line(text):
    translations = {'English': text}
    for lang, translator in translation_pipelines.items():
        try:
            translated = translator(text, max_length=512, truncation=True)[0]['translation_text']
        except Exception as e:
            print(f"Error translating '{text[:30]}...' to {lang}: {e}")
            translated = ""
        translations[lang] = translated
    return translations

def split_file(input_file, lines_per_file=100000):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    lines = [line.strip() for line in lines if line.strip()]
    total_parts = (len(lines) + lines_per_file - 1) // lines_per_file

    for part_num in range(total_parts):
        part_lines = lines[part_num * lines_per_file : (part_num + 1) * lines_per_file]
        part_filename = f'/content/split_files/DS1split_part_{part_num+1}.txt'
        with open(part_filename, 'w', encoding='utf-8') as part_file:
            part_file.write('\n'.join(part_lines))
        print(f"✅ Saved split: {part_filename} ({len(part_lines)} lines)")

def log_completed_split(split_name):
    completed_log = '/content/translated/log_completed.txt'

    with open(completed_log, 'a') as f:
        f.write(split_name + '\n')
    print(f"✅ Logged completed split: {split_name}")

def translate_file(split_file_path, batch_size=100):
    base_name = os.path.basename(split_file_path).replace('.txt', '')
    checkpoint_csv_path = f'/content/drive/MyDrive/translation_checkpoints/{base_name}_checkpoint.csv'
    output_csv_path = f'/content/translated/DS1{base_name}_translated.csv'

    # Check if already translated (based on output CSV)
    if os.path.exists(output_csv_path):
        print(f"⏩ Skipping {split_file_path} — already translated.")
        os.remove(split_file_path)  # Optional cleanup
        return

    # Create a log file to track completed translations
    completed_log = '/content/translated/log_completed.txt'
    if not os.path.exists(completed_log):
        open(completed_log, 'w').close()

    with open(completed_log, 'r') as f:
        completed_parts = set(line.strip() for line in f.readlines())

    # Skip if already completed (check log)
    if base_name in completed_parts:
        print(f"⏩ Skipping {split_file_path} — already completed.")
        os.remove(split_file_path)  # Optional cleanup
        return

    with open(split_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    lines = [line.strip() for line in lines if line.strip()]

    # Check if resuming from checkpoint
    if os.path.exists(checkpoint_csv_path):
        translated_df = pd.read_csv(checkpoint_csv_path)
        start_from_line = len(translated_df)
        print(f"🔁 Resuming from line {start_from_line}")
    else:
        translated_df = pd.DataFrame()
        start_from_line = 0

    for batch_start in range(start_from_line, len(lines), batch_size):
        batch = lines[batch_start : batch_start + batch_size]
        batch_translations = []

        for line in batch:
            result = translate_line(line)
            batch_translations.append(result)

        batch_df = pd.DataFrame(batch_translations)
        translated_df = pd.concat([translated_df, batch_df], ignore_index=True)

        # Save checkpoint after each batch
        translated_df.to_csv(checkpoint_csv_path, index=False, encoding='utf-8-sig')
        print(f"✅ Translated {len(translated_df)} / {len(lines)} lines for {split_file_path}")

    # Final save
    translated_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"🎯 Finished translation: {output_csv_path}")

    # Clean up files
    os.remove(split_file_path)
    print(f"🗑️ Deleted split file to save space: {split_file_path}")

    # Log completion of this part
    log_completed_split(base_name)

    if os.path.exists(checkpoint_csv_path):
        os.remove(checkpoint_csv_path)
        print(f"🗑️ Deleted checkpoint: {checkpoint_csv_path}")

# CHANGE this to your real input file path
input_file_path = 'en.txt'  # <-- CHANGE this!
lines_per_split = 1000  # Split every 100,000 lines (adjust if needed)
batch_size = 100  # Translate 100 lines at a time (adjust if needed)

# Step 1: Split the big file
print("🔵 Splitting large file...")
split_file(input_file_path, lines_per_file=lines_per_split)

# Step 2: Translate each smaller file
print("🔵 Translating each chunk...")
split_files = sorted(glob.glob('/content/split_files/*.txt'))

for split_file_path in split_files:
    print(f"🔵 Now translating: {split_file_path}")
    translate_file(split_file_path, batch_size=batch_size)

print("🏆 All translations completed successfully!")


🔵 Loading translation models...


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


Streaming output truncated to the last 5000 lines.
✅ Saved split: /content/split_files/DS1split_part_1131258.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131259.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131260.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131261.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131262.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131263.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131264.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131265.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131266.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131267.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131268.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part_1131269.txt (1000 lines)
✅ Saved split: /content/split_files/DS1split_part

KeyboardInterrupt: 